**Install External Software**

In [ ]:
try:
    import ta
    print("Package 'ta' is already installed.")
except ImportError:
    !pip install ta

try:
    import lightweight_charts
    print("Package 'lightweight_charts' is already installed.")
except ImportError:
    !pip install lightweight_charts


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=adadb8e6d58dcb03bb133f91138e5258f13e4e39f4d6ba1fb5914a27a0ef8639
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.7 MB/s eta 0:00:00
  Created wheel for proxy_tools: filename=proxy_tools-0.1.0-py3-none-any.whl size=2901 sha256=c0832a4770b74d76798c17762188224ffc415271633535e8caafcc66d5474417
  Stored in directory: /root/.cache/pip/wheels/07/37/12/f3390260c831aa68c3835d6ec2e10d8da094b5fc0ac32feb14
Successfully built proxy_tools


**Clone Repository and Reload**

In [ ]:
#Clone Repository and Reload
import os, subprocess, sys
import sys, importlib
code_dir = "/content/pyfx_utils"  # <- your repo root
if code_dir not in sys.path:
    sys.path.insert(0, code_dir)

if not os.path.exists("pyfx_utils"): subprocess.run(["git", "clone", "--branch", "main", "https://github.com/jbaird123/pyfx_utils.git", "pyfx_utils"])
subprocess.run(["git", "-C", "pyfx_utils", "pull", "origin", "2025-0904-re-org"])

importlib.invalidate_caches()
try:
    %unload_ext pyfx_utils.ipyext
except Exception:
    pass
%reload_ext pyfx_utils.ipyext
%pyfx_boot

The pyfx_utils.ipyext extension is not loaded.
Mounted at /content/drive
✅ pyfx booted: BTConfig, StrategyRunMeta, add_indicators, add_sma, add_trades, add_y_padding, annotate_trades, backtest, backtest_signal, build_ohlc, build_pips_brief, compute_pips, equity_curve_from_signal, equity_curve_from_trades, infer_pip_size, load_fx_csv, metrics, metrics_by_period, new_chart, prepare_trades_for_chart, resample, trade_pnls, walk_forward_ranges


**Mount Google Drive**

In [ ]:
#mount google drive and import libraries
from google.colab import drive; drive.mount("/content/drive")
import sys
import os as os
sys.path.insert(0, '/content/drive/MyDrive/fx/code')
os.environ["PYFX_OUTDIR"] = "/content/drive/MyDrive/fx/output"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Imports**

In [ ]:

# Technical Analysis
import ta

# Imports
import pandas as pd

# Strategy + backtest
from pyfx_utils.strategies.sma import SMACrossoverStrategy, SMACrossoverAdapter                     # your SMA strategy file
from pyfx_utils.backtests.core import backtest

# Brief building + extras
from pyfx_utils.analysis.interfaces import StrategyRunMeta
from pyfx_utils.analysis.augment import extend_brief_with_analyses,annotate_trades,build_pips_brief
from pyfx_utils.analysis.tuning import (
    obj_total_pips,      # objective choices
    obj_total_minus_half_dd,
    obj_pips_over_dd,
)

# Simple helpers we moved into the package (see patch file below)
from pyfx_utils.utils.stats import infer_pip_size, compute_pips


**Load Raw File**

In [ ]:
#FILE = "/content/drive/MyDrive/fx/backtesting_data/sample_eurusd_minute.csv"
FILE = "/content/drive/MyDrive/fx/backtesting_data/eurusd_interval_minute_1_20130101_20250823.csv"

try:
    raw  # Check if raw is defined
    print("Variable 'raw' is already defined.")
except NameError:
    raw = load_fx_csv(FILE)

        timezone: UTC
            rows: 4911893
       dup_index: 0
           nulls: 0
       top_freqs: {Timedelta('0 days 00:01:00'): 4899165, Timedelta('0 days 00:02:00'): 9522, Timedelta('0 days 00:03:00'): 1529}
 ohlc_violations: 0
    weekend_rows: 288562
ℹ️ Weekend rows present. Confirm your broker's session conventions.


**Resample and Execute Strategy**

In [ ]:

from pyfx_utils.analysis import annotate_trades

# Resample as needed
df = resample(raw, "1D")

symbol = "EUR/USD"
pip = infer_pip_size(symbol)

strategy = SMACrossoverStrategy(fast=10, slow=20)

# Backtest (pips per trade only)
trades = backtest(df, strategy, pip=pip)

# Define additional indicators to add
df = add_indicators(df, pip=pip, include=("rsi","adx","bb_width","atr_pips"))

trades = annotate_trades(
    trades=trades,
    data=df,
    symbol=symbol,
    price_col="close",
    feature_cols=("rsi","adx","atr_pips","bb_width"),
    at="entry",
    strict_features=True,        # set True if df definitely has those feature columns
)

print(f"Total trades: {len(trades)}")
df.tail()


Total trades: 223


,open,high,low,close,volume,rsi,adx,bb_width,atr_pips
timestamp,,,,,,,,,
2025-08-19 00:00:00+00:00,1.17071,1.17106,1.16561,1.16676,67120.690,51.850520,13.116485,0.028856,70.449154
2025-08-20 00:00:00+00:00,1.16675,1.16928,1.16390,1.16420,67270.940,49.401273,13.463731,0.026880,69.259929
2025-08-21 00:00:00+00:00,1.16421,1.16736,1.16221,1.16508,72388.610,50.270870,14.013971,0.026940,67.991363
2025-08-22 00:00:00+00:00,1.16509,1.16626,1.16010,1.16122,89596.770,46.496153,14.804661,0.026416,67.534837
2025-08-23 00:00:00+00:00,1.16121,1.17425,1.15828,1.17148,100705.956,55.961656,14.033672,0.022579,74.118063


**Add Technical Indicators and Create Brief**

In [ ]:
# Define the strategy and run a base backtest

strat = SMACrossoverStrategy(
    fast=10,
    slow=50,
    stop_type="atr",
    stop_value=2,
    tp_type="atr",
    tp_value=4,
    trail_type=None
)

# Convert your strategy settings to a params dict
params = {
    "fast": strat.fast, "slow": strat.slow,
    "stop_type": strat.stop_type, "stop_value": strat.stop_value,
    "tp_type": strat.tp_type, "tp_value": strat.tp_value,
    "trail_type": strat.trail_type, "trail_value": getattr(strat, "trail_value", None),
    "atr_length": strat.atr_length, "tie_break": strat.tie_break,
}

# Required meta fields (YOUR dataclass needs run_id and timestamp)
ts = pd.Timestamp.now(tz='UTC')
meta = StrategyRunMeta(
    strategy_name="SMACrossoverStrategy",
    universe=symbol,          # if your meta uses 'universe' instead of 'symbol', switch the name
    timeframe="D",
    version="1.0.0",
    run_id=f"sma_{symbol}_{ts:%Y%m%d_%H%M%S}",
    timestamp=ts,
)

# Use your trades
adapter = SMACrossoverAdapter(
    trades_df=trades,
    params=params,
    meta=meta,
    bar_index=df.index,
)
payload = adapter.to_payload()

# build_pips_brief expects a keyword
brief = build_pips_brief(payload=payload)


**Extend Brief with Additional Analysis**

In [ ]:
# Define your tuning search space (all values as sequences/lists)
# NOTE: every value must be a list/sequence. No bare strings/numbers.
param_space = {
    "fast": [10, 15, 20, 25],
    "slow": [30, 35, 40, 45, 50, 55, 60, 65, 70],
    "stop_type": ["atr"],  #["pips", "atr", None],
    "stop_value": [1.5, 2.0, 2.5, 3.0],
    "tp_type": ["atr"],    #["pips", "atr", None],
    "tp_value": [3.0, 4.0, 6.0, 8.0],
    # Optional trailing examples if you enable trailing in your core:
    # "trail_type": [None, "pips", "atr"],
    # "trail_value": [None, 100, 1.5],
    # "trail_mode": ["chandelier"],
    # "tie_break": ["stop_then_tp"],
}


# One call to produce all the extra analyses (tuning, WF, stationarity, regimes)
extra = extend_brief_with_analyses(
    df=df,
    strategy_cls=SMACrossoverStrategy,
    params_space=param_space,
    pip=pip,
    when="exit",       # cumulative pips measured at exits
    n_splits=4,        # walk-forward folds
    n_trials=80,       # used only if the grid would be too big; otherwise grid search is used
    objective=obj_total_pips,  # swap to another objective if you want
)

# Merge into the brief in a couple lines
brief["tuning"]            = extra["param_search"]
brief["walk_forward"]      = extra["walk_forward"]
brief["stationarity"]      = extra["stationarity"]
brief["regimes"]           = extra["regimes"]
brief["tuning_objective"]  = extra["objective"]


**1) Fix the tuning dict inconsistencies (None + value)**

In [ ]:
from pyfx_utils.analysis.tuning import run_param_search
from pyfx_utils.analysis.augment import build_pips_brief
from pyfx_utils.analysis.interfaces import StrategyRunPayload
from pyfx_utils.analysis.regime import build_regime_features, kmeans_regimes
# (Adjust imports to match your actual paths.)

# 1) Define a strategy runner that returns a trades DataFrame with 'pips'
def run_sma_strategy(data, params):
    """
    Run the SMA strategy with given params on an OHLCV DataFrame.
    Must return a trades DataFrame with 'pips' column.
    """
    strat = SMACrossoverStrategy(**params)
    pip = infer_pip_size(params.get("symbol", "EUR/USD"))  # or hardcode if you know
    trades = backtest(data, strat, pip=pip)
    return trades


# 2) Define a parameter grid (None allowed)
param_grid = {
    "fast": [10, 20],
    "slow": [50],
    "stop_type": [None, "atr"],
    "stop_value": [None, 2],
    "tp_type": [None, "atr"],
    "tp_value": [None, 4],
    # add anything else you sweep over...
}

# 3) Execute the grid search
tuning = run_param_search(
    data=df,
    strategy_runner=run_sma_strategy,
    param_grid=param_grid,
    # Optional: custom ranking function. Example: prefer higher mean, then lower DD
    # score_fn=lambda m: (m["mean_pips"], -m["max_dd_pips"]),
    top_n=25,
)

best_params = tuning.get("best_params", {}) or params  # fallback to current params
trades_best = run_sma_strategy(df, best_params)

# Build features from your OHLCV dataframe
features = build_regime_features(df)

# Cluster into regimes (k=3, change if you like)
regime_series = kmeans_regimes(features, k=3)  # -> pd.Series indexed like df


# 4) Attach tuning to your brief (no _jsonify in the notebook; the package handles it)
payload = StrategyRunPayload(
    trades=trades,
    meta=meta,
    params=params,
    bar_index=df.index,
    regime_series=regime_series,          # optional
    regimes_meta={"k": 3},                # optional
    active_regime=None                    # optional (auto-picks if None)
)

brief = build_pips_brief(payload)
brief["tuning"] = tuning


**12) Final: make it JSON (double quotes), safely**

In [ ]:
import json
from pyfx_utils.analysis.augment import _jsonify

brief_json = json.dumps(_jsonify(brief), indent=2)
print(brief_json)


**Chart It!**

In [ ]:
import json
import numpy as np
import pandas as pd
from IPython.display import HTML, display

# --- strategy inspection: infer what it uses based on attributes ---
def _detect_from_strategy(strategy):
    def has(n): return hasattr(strategy, n)
    def get(n, d=None): return getattr(strategy, n, d)

    features = {
        "ma": None,   # {'type':'sma'|'ema','fast':int,'slow':int}
        "bb": None,   # {'length':int,'mult':float}
        "atr": None,  # {'length':int}
        "rsi": None,  # {'length':int}
        "adx": None,  # {'length':int}
        "macd": None, # {'fast':int,'slow':int,'signal':int}
    }
    if has("fast") and has("slow"):
        ma_type = str(get("ma_type", "sma")).lower()
        if ma_type not in ("sma","ema"):
            ma_type = "sma"
        features["ma"] = {"type": ma_type, "fast": int(get("fast")), "slow": int(get("slow"))}
    if has("bb_length") and has("bb_mult"):
        features["bb"] = {"length": int(get("bb_length")), "mult": float(get("bb_mult"))}
    elif has("bb_window") and has("bb_k"):
        features["bb"] = {"length": int(get("bb_window")), "mult": float(get("bb_k"))}
    stop_tp_trail_uses_atr = any(str(get(x, None)).lower() == "atr" for x in ("stop_type","tp_type","trail_type"))
    if stop_tp_trail_uses_atr or has("atr_length"):
        features["atr"] = {"length": int(get("atr_length", 14))}
    if hasattr(strategy, "rsi_length"):
        features["rsi"] = {"length": int(get("rsi_length", 14))}
    if hasattr(strategy, "adx_length"):
        features["adx"] = {"length": int(get("adx_length", 14))}
    if all(has(k) for k in ("macd_fast","macd_slow","macd_signal")):
        features["macd"] = {"fast": int(get("macd_fast")), "slow": int(get("macd_slow")), "signal": int(get("macd_signal"))}
    uses = getattr(strategy, "uses_indicators", None)
    if uses:
        u = {s.lower() for s in uses}
        if "rsi" in u and features["rsi"] is None: features["rsi"] = {"length": 14}
        if "adx" in u and features["adx"] is None: features["adx"] = {"length": 14}
        if "atr" in u and features["atr"] is None: features["atr"] = {"length": int(get("atr_length", 14))}
        if "bb"  in u and features["bb"]  is None: features["bb"]  = {"length": 20, "mult": 2.0}
        if "ema" in u and features["ma"]  is None and hasattr(strategy,"fast") and hasattr(strategy,"slow"):
            features["ma"] = {"type":"ema","fast":int(get("fast")),"slow":int(get("slow"))}
    return features

def plot_strategy_inline_autodetect(
    df: pd.DataFrame,
    strategy,
    trades_df: pd.DataFrame | None = None,
    *,
    show_volume: bool = True,
    title: str = "Strategy Chart",
    height: int = 720,
):
    """
    Strategy-aware inline chart (lightweight-charts via JS, no GTK/Qt).
    - Overlays (SMA/EMA, BB) on price with distinct colors & thin lines.
    - Each non-overlay indicator gets its own panel (Volume, ATR, RSI, ADX, MACD).
    - Trades drawn as markers + green/red connector lines.
    """
    # ---------- normalize time & OHLC ----------
    d = df.copy()
    if "time" in d.columns:
        t = pd.to_datetime(d["time"], utc=True)
    elif isinstance(d.index, pd.DatetimeIndex):
        t = pd.to_datetime(d.index, utc=True)
    else:
        for k in ("date","datetime","timestamp"):
            if k in d.columns:
                t = pd.to_datetime(d[k], utc=True)
                break
        else:
            raise ValueError("df needs datetime index or a 'time'/'date'/'datetime'/'timestamp' column.")
    tsecs = (t.astype("int64") // 10**9).astype("int64")

    cols = {c.lower(): c for c in d.columns}
    for need in ("open","high","low","close"):
        if need not in cols:
            raise ValueError("df must contain OHLC columns (case-insensitive).")
    o = pd.to_numeric(d[cols["open"]],  errors="coerce")
    h = pd.to_numeric(d[cols["high"]],  errors="coerce")
    l = pd.to_numeric(d[cols["low"]],   errors="coerce")
    c = pd.to_numeric(d[cols["close"]], errors="coerce")

    ohlc_records = pd.DataFrame({"time": tsecs, "open": o, "high": h, "low": l, "close": c}).dropna().to_dict("records")

    # ---------- compute ONLY what the strategy uses (via ta) ----------
    from ta.trend import SMAIndicator, EMAIndicator, ADXIndicator, MACD
    from ta.momentum import RSIIndicator
    from ta.volatility import BollingerBands, AverageTrueRange  # (ATR is in volatility in 'ta')

    feats = _detect_from_strategy(strategy)

    # Overlays for price pane
    overlays = []  # list of {name, data}
    if feats["ma"] is not None:
        m = feats["ma"]; f, s = m["fast"], m["slow"]
        if m["type"] == "ema":
            overlays.append({"name": f"EMA {f}", "data": pd.DataFrame({"time": tsecs, "value": EMAIndicator(close=c, window=f).ema_indicator()}).dropna().to_dict("records")})
            overlays.append({"name": f"EMA {s}", "data": pd.DataFrame({"time": tsecs, "value": EMAIndicator(close=c, window=s).ema_indicator()}).dropna().to_dict("records")})
        else:
            overlays.append({"name": f"SMA {f}", "data": pd.DataFrame({"time": tsecs, "value": SMAIndicator(close=c, window=f).sma_indicator()}).dropna().to_dict("records")})
            overlays.append({"name": f"SMA {s}", "data": pd.DataFrame({"time": tsecs, "value": SMAIndicator(close=c, window=s).sma_indicator()}).dropna().to_dict("records")})
    if feats["bb"] is not None:
        L, K = feats["bb"]["length"], feats["bb"]["mult"]
        bb = BollingerBands(close=c, window=L, window_dev=K)
        overlays.append({"name": f"BB mid {L}",   "data": pd.DataFrame({"time": tsecs, "value": bb.bollinger_mavg()}).dropna().to_dict("records")})
        overlays.append({"name": f"BB upper {L}", "data": pd.DataFrame({"time": tsecs, "value": bb.bollinger_hband()}).dropna().to_dict("records")})
        overlays.append({"name": f"BB lower {L}", "data": pd.DataFrame({"time": tsecs, "value": bb.bollinger_lband()}).dropna().to_dict("records")})

    # Panels (each indicator gets its own)
    panels = []  # each: {title, type:'lines'|'hist'|'mix', series:[{kind:'line'|'hist', name, data, extra?}]}
    # Volume
    if show_volume and "volume" in cols:
        v = pd.to_numeric(d[cols["volume"]], errors="coerce")
        vrec = pd.DataFrame({"time": tsecs, "value": v}).dropna().to_dict("records")
        if vrec:
            panels.append({
                "title": "Volume",
                "type": "hist",
                "series": [ {"kind":"hist", "name":"Volume", "data": vrec, "opts": {"priceFormat":{"type":"volume"}} } ]
            })
    # ATR
    if feats["atr"] is not None:
        alen = feats["atr"]["length"]
        atr = AverageTrueRange(high=h, low=l, close=c, window=alen).average_true_range()
        arec = pd.DataFrame({"time": tsecs, "value": atr}).dropna().to_dict("records")
        if arec:
            panels.append({
                "title": f"ATR {alen}",
                "type": "lines",
                "series": [ {"kind":"line", "name": f"ATR {alen}", "data": arec} ]
            })
    # RSI
    if feats["rsi"] is not None:
        rlen = feats["rsi"]["length"]
        rsi = RSIIndicator(close=c, window=rlen).rsi()
        rrec = pd.DataFrame({"time": tsecs, "value": rsi}).dropna().to_dict("records")
        if rrec:
            panels.append({
                "title": f"RSI {rlen}",
                "type": "lines",
                "series": [ {"kind":"line", "name": f"RSI {rlen}", "data": rrec, "bands":[30,70]} ]
            })
    # ADX
    if feats["adx"] is not None:
        adlen = feats["adx"]["length"]
        adx = ADXIndicator(high=h, low=l, close=c, window=adlen).adx()
        adrec = pd.DataFrame({"time": tsecs, "value": adx}).dropna().to_dict("records")
        if adrec:
            panels.append({
                "title": f"ADX {adlen}",
                "type": "lines",
                "series": [ {"kind":"line", "name": f"ADX {adlen}", "data": adrec} ]
            })
    # MACD (lines + hist in one panel)
    if feats["macd"] is not None:
        mf, ms, sig = feats["macd"]["fast"], feats["macd"]["slow"], feats["macd"]["signal"]
        macd = MACD(close=c, window_fast=mf, window_slow=ms, window_sign=sig)
        macd_line  = pd.DataFrame({"time": tsecs, "value": macd.macd()}).dropna()
        signal_line= pd.DataFrame({"time": tsecs, "value": macd.macd_signal()}).dropna()
        hist       = pd.DataFrame({"time": tsecs, "value": macd.macd_diff()}).dropna()
        if not macd_line.empty:
            hrec = hist.copy()
            hrec["color"] = np.where(hrec["value"]>=0, "#0ECB81", "#EF5350")
            panels.append({
                "title": f"MACD {mf}-{ms} / Signal {sig}",
                "type": "mix",
                "series": [
                    {"kind":"line","name":f"MACD {mf}-{ms}", "data": macd_line.to_dict("records")},
                    {"kind":"line","name":f"Signal {sig}",   "data": signal_line.to_dict("records")},
                    {"kind":"hist","name":"MACD hist",       "data": hrec.to_dict("records")}
                ]
            })

    # Trades (markers + connector lines)
    trades = []
    if trades_df is not None and len(trades_df):
        td = trades_df.copy()
        for k in ("entry_time","exit_time"):
            td[k] = pd.to_datetime(td[k], utc=True)
        trades = [{
            "entry_time": int(et.value // 10**9),
            "exit_time":  int(xt.value // 10**9),
            "entry_price": float(row["entry_price"]),
            "exit_price":  float(row["exit_price"]),
            "side":        str(row.get("side","")).lower(),
            "win":         float(row.get("pips", 0.0)) >= 0.0,
        } for et, xt, row in zip(td["entry_time"], td["exit_time"], td.to_dict("records"))]

    # --- layout: price gets ~62%, remaining split across panels equally ---
    n_panels = len(panels)
    price_h = int(height * (0.62 if n_panels else 1.0))
    rem = max(0, height - price_h)
    panel_heights = [ int(rem / n_panels) ] * n_panels if n_panels else []

    payload = {
        "title": title,
        "price_h": price_h,
        "panel_heights": panel_heights,
        "ohlc": ohlc_records,
        "overlays": overlays,      # [{name, data}]
        "panels": panels,          # [{title, type, series:[{kind,name,data,(opts|bands)?}]}]
        "trades": trades,
        "height": height,
    }
    js = json.dumps(payload)

    html = f"""
<div id="lwc-wrap" style="width:100%; max-width:1200px;">
  <h3 style="margin:6px 0 8px 0; font-family:system-ui, -apple-system, Segoe UI, Roboto;">{title}</h3>
  <div id="chart-price" style="height:{price_h}px;"></div>
  <div id="chart-panels"></div>
</div>
<script src="https://unpkg.com/lightweight-charts@4.2.0/dist/lightweight-charts.standalone.production.js"></script>
<script>
(function() {{
  const P = {js};
  const gridColor = 'rgba(197,203,206,0.3)', green = '#0ECB81', red = '#EF5350';

  // --- helpers ---
  function makeChart(el, h) {{
    return LightweightCharts.createChart(el, {{
      height: h,
      layout: {{ background: {{ type: 'solid', color: '#ffffff' }}, textColor: '#333' }},
      grid: {{ vertLines: {{ color: gridColor }}, horzLines: {{ color: gridColor }} }},
      rightPriceScale: {{ borderColor: '#D1D4DC' }},
      timeScale: {{ rightOffset: 5, borderColor: '#D1D4DC' }},
      crosshair: {{ mode: 1 }},
    }});
  }}
  // simple palette for overlays (distinct colors, thin lines)
  const overlayPalette = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#17becf','#8c564b','#e377c2','#7f7f7f','#bcbd22'];

  // --- main (price) chart ---
  const priceEl = document.getElementById('chart-price');
  const priceChart = makeChart(priceEl, P.price_h);
  const candles = priceChart.addCandlestickSeries();
  candles.setData(P.ohlc);

  // overlays as thin colored lines
  (P.overlays || []).forEach((ov, idx) => {{
    const color = overlayPalette[idx % overlayPalette.length];
    const line = priceChart.addLineSeries({{
      title: ov.name, lineWidth: 1, color: color,
      priceLineVisible:false, lastValueVisible:false,
    }});
    line.setData(ov.data);
    // Bollinger mid/upper/lower: soften upper/lower a bit
    if (ov.name.toLowerCase().includes('bb upper')) line.applyOptions({{ color: 'rgba(31,119,180,0.35)' }});
    if (ov.name.toLowerCase().includes('bb lower')) line.applyOptions({{ color: 'rgba(31,119,180,0.35)' }});
    if (ov.name.toLowerCase().includes('bb mid'))   line.applyOptions({{ color: 'rgba(100,100,100,0.6)' }});
  }});

  // trades: markers + connector lines per trade
  const markers = [];
  (P.trades || []).forEach(tr => {{
    const col = tr.win ? green : red;
    if (tr.side === 'long') {{
      markers.push({{ time: tr.entry_time, position: 'belowBar', color: green, shape: 'arrowUp', text: 'L' }});
      markers.push({{ time: tr.exit_time,  position: 'aboveBar', color: col,   shape: 'arrowDown', text: 'x' }});
    }} else if (tr.side === 'short') {{
      markers.push({{ time: tr.entry_time, position: 'aboveBar', color: red,   shape: 'arrowDown', text: 'S' }});
      markers.push({{ time: tr.exit_time,  position: 'belowBar', color: col,   shape: 'arrowUp', text: 'x' }});
    }} else {{
      markers.push({{ time: tr.entry_time, position: 'inBar', color: col, shape: 'circle', text: '•' }});
      markers.push({{ time: tr.exit_time,  position: 'inBar', color: col, shape: 'square', text: '■' }});
    }}
    const ls = priceChart.addLineSeries({{
      color: col, lineWidth: 2, lastValueVisible:false, priceLineVisible:false, crosshairMarkerVisible:false
    }});
    ls.setData([{{ time: tr.entry_time, value: tr.entry_price }},
                {{ time: tr.exit_time,  value: tr.exit_price  }}]);
  }});
  candles.setMarkers(markers);

  // --- panels: one chart per non-overlay indicator ---
  const panelWrap = document.getElementById('chart-panels');
  const charts = [priceChart];  // for syncing
  (P.panels || []).forEach((pane, i) => {{
    const div = document.createElement('div');
    div.style.height = (P.panel_heights[i] || 140) + 'px';
    div.style.marginTop = '8px';
    panelWrap.appendChild(div);

    const ch = makeChart(div, (P.panel_heights[i] || 140));
    charts.push(ch);

    // add series
    (pane.series || []).forEach(s => {{
      if (s.kind === 'hist') {{
        const hist = ch.addHistogramSeries(Object.assign({{ title: s.name, priceLineVisible:false }}, s.opts || {{}}));
        hist.setData(s.data);
        if (!s.opts || !s.opts.priceFormat) {{
          // default compact scale for non-volume hists too
          hist.priceScale().applyOptions({{ scaleMargins: {{ top: 0.2, bottom: 0.05 }} }});
        }}
      }} else {{
        const ln = ch.addLineSeries({{ title: s.name, lineWidth: 2, priceLineVisible:false }});
        ln.setData(s.data);
        // RSI helper bands (30/70)
        if (s.bands && Array.isArray(s.bands)) {{
          const times = s.data.map(p => p.time);
          s.bands.forEach(v => {{
            const band = ch.addLineSeries({{ color: 'rgba(180,180,180,0.4)', lineWidth:1, lastValueVisible:false, priceLineVisible:false }});
            band.setData(times.map(t => ({{ time: t, value: v }})));
          }});
        }}
      }}
    }});
  }});

  // --- visible range sync across all charts ---
  function syncVisible(src) {{
    src.timeScale().subscribeVisibleTimeRangeChange(range => {{
      if (!range) return;
      charts.forEach(ch => {{ if (ch !== src) ch.timeScale().setVisibleRange(range); }});
    }});
  }}
  charts.forEach(ch => syncVisible(ch));

  // fit views
  charts.forEach(ch => ch.timeScale().fitContent());
}})();
</script>
"""
    display(HTML(html))


In [ ]:
# df: your OHLC(+volume) DataFrame; strategy: your Strategy instance; trades: trade results DataFrame
plot_strategy_inline_autodetect(
    df=df,
    strategy=strategy,
    trades_df=trades,
    show_volume=True,
    title="EUR/USD — Strategy-aware Plot"
)
